In [1]:
import os
import re
from flair.data import Sentence
from flair.models import SequenceTagger
import unicodedata

In [2]:
#Modèle Flair
tagger = SequenceTagger.load("/Users/pauline/Documents/Sandoz/pliegos-ner/final-model.pt")

2024-02-16 12:22:29,427 SequenceTagger predicts: Dictionary with 5 tags: O, S-LOC, B-LOC, E-LOC, I-LOC


In [3]:
folder = '/Users/pauline/Documents/Sandoz/pliegos-ner/Varios-ner/Data/TEXT'
output_folder = '/Users/pauline/Documents/Sandoz/pliegos-ner/Varios-ner/Data/IOB'

In [4]:
if not os.path.exists(output_folder):
    os.makedirs(output_folder)

In [5]:
for f in os.listdir(folder):
    if f.endswith('.txt'):
        txt_path = os.path.join(folder, f)
        label, ext = os.path.splitext(f)

        with open(txt_path, encoding='utf-8') as t:
            text = t.read()
            # Normalisation NFC
            text_normalized = unicodedata.normalize('NFC', text)
            sentence = Sentence(text_normalized)
            tagger.predict(sentence)

            output = ""        
            for entity in sentence.get_spans('ner'):
                prefix = 'B-'
                for token in entity:
                    token.set_label('ner-bio', prefix + entity.tag)
                    prefix = 'I-'

            for token in sentence:
                output += str(token.get_label('ner-bio'))
                output += "\n"

            result = re.sub(r"([A-Z]\w+\[[0-9]+\]:\s\")([A-Z a-z 0-9 .,?!:¿¡…\'\*´ `’‘℣Đ=æǝ°>\—\-\_–\(\)\¬\«\»;~óòáàâíìñéèúüùäëïöÀÁÉÈÊËÍÌÎÏÑÓÒÔÖÚÙÛÜ\"\"]+)(\"\s→\s)([A-Z-]*)(\s\([0-9 .]*\))", r"\2 \4", output)

            output_file_name = f'Ner_{label}.txt'
            with open(os.path.join(output_folder, output_file_name), 'w', encoding='utf-8') as x:
                x.write(result)